# ETL 

**Camada Bronze (aac_intakes e aac_outcomes)**

In [0]:
-- Criação da camada bronze: carregar e armazenar os dados brutos de aac_intakes e aac_outcomes

DROP TABLE IF EXISTS bronze_aac_intakes;
DROP TABLE IF EXISTS bronze_aac_outcomes;
CREATE TABLE bronze_aac_intakes AS
SELECT * FROM aac_intakes;
CREATE TABLE bronze_aac_outcomes AS
SELECT * FROM aac_outcomes;

-- Checagem de carregamento de dados

SELECT count(*) FROM bronze_aac_intakes;
SELECT count(*) FROM bronze_aac_outcomes;

**Camada Prata - (aac_intakes)**

In [0]:
-- Criação camada prata: limpar e padronizar dados 

-- Base aac_intakes

DROP TABLE IF EXISTS prata_aac_intakes;
CREATE TABLE prata_aac_intakes AS
SELECT DISTINCT
animal_id as id,

CASE
  WHEN name IS NULL THEN 'Unknown'
  WHEN name like '%*%' THEN REPLACE (name,'*','')
  ELSE name
END as nome_animal,

animal_type as especie,
breed as raca,
color as cor,
age_upon_intake as idade_entrada,

cast(datetime as date) as data_entrada, 

found_location as local_encontrado,
intake_condition as condicao_entrada,
intake_type as tipo_entrada,

CASE
  WHEN sex_upon_intake IS NULL THEN 'Unknown'
  WHEN lower(sex_upon_intake) like '%female%' THEN 'Female'
  WHEN lower(sex_upon_intake) LIKE '%male%' THEN 'Male'
  ELSE 'Unknown'
END AS sexo,

CASE
  WHEN sex_upon_intake IS NULL THEN 'Unknown'
  WHEN lower(sex_upon_intake) LIKE '%spayed%' THEN 'Yes'
  WHEN lower(sex_upon_intake) LIKE '%neutered%' THEN 'Yes'
  WHEN lower(sex_upon_intake) LIKE '%intact%' THEN 'No'
  ELSE 'Unknown'
END AS castrado

FROM bronze_aac_intakes;

-- Usei o "SELECT DISTINCT" para remover valores duplicados. 
-- Usei o "AS" para renomear as colunas da base original com nomes mais simples e adequados.
-- Tratei a coluna name, uma vez que na base original alguns nomes continham *. Para isso usei o "REPLACE" 
-- Mantive apenas uma coluna de data (data_entrada). A base original tinha duas colunas com a mesma informação. Como a coluna de data vinha no formato datatime (data e hora), usei "CAST" para mudar o formato para data (trazendo apenas datas)
-- Tratei a coluna "sex_upon_intake" para trazer de forma separada as informações de sexo e castração do animal. Para isso usei "CASE".


**Camada Prata - (aac_outcomes)**

In [0]:
DROP TABLE IF EXISTS prata_aac_outcomes;
CREATE TABLE prata_aac_outcomes AS
SELECT DISTINCT
animal_id as id,

CASE
  WHEN name IS NULL THEN 'Unknown'
  WHEN name like '%*%' THEN REPLACE (name,'*','')
  ELSE name
END as nome_animal,

animal_type as especie,
breed as raca,
color as cor,
age_upon_outcome as idade_saida,

cast(date_of_birth as date) as data_nascimento, 

cast(datetime as date) as data_saida, 

CASE
  WHEN outcome_type IS NULL THEN 'Unknown'
  WHEN outcome_type = 'Rto-Adopt' THEN 'Return to Owner'
  WHEN outcome_type = 'Relocate' THEN 'Transfer'
  WHEN outcome_type = 'Disposal' THEN 'Euthanasia'
  ELSE outcome_type
END AS tipo_saida,

CASE
  WHEN outcome_subtype IS NULL THEN 'Unknown'
  WHEN outcome_subtype = 'In Foster' THEN 'Foster'
  WHEN outcome_subtype = 'Aggressive' THEN 'Behavior'
  WHEN outcome_subtype IN ('SCRP', 'Suffering', 'Rabies Risk', 'Snr', 'At Vet', 'In Surgery') THEN 'Medical'  
  WHEN outcome_subtype IN ('Court/Investigation', 'Possible Theft') THEN 'Investigation'  
  ELSE outcome_subtype  
END AS subtipo_saida,

CASE
  WHEN sex_upon_outcome IS NULL THEN 'Unknown'
  WHEN lower(sex_upon_outcome) like '%female%' THEN 'Female'
  WHEN lower(sex_upon_outcome) LIKE '%male%' THEN 'Male'
  ELSE 'Unknown'
END AS sexo,

CASE
  WHEN sex_upon_outcome IS NULL THEN 'Unknown'
  WHEN lower(sex_upon_outcome) LIKE '%spayed%' THEN 'Yes'
  WHEN lower(sex_upon_outcome) LIKE '%neutered%' THEN 'Yes'
  WHEN lower(sex_upon_outcome) LIKE '%intact%' THEN 'No'
  ELSE 'Unknown'
END AS castrado

FROM bronze_aac_outcomes;

-- Usei o "SELECT DISTINCT" para remover valores duplicados. 
-- Usei o "AS" para renomear as colunas da base original com nomes mais simples e adequados.
-- Tratei a coluna name, uma vez que na base original alguns nomes continham *.Para isso usei REPLACE
-- Transformei a coluna de data (data_saida), que vinha no formato datetime (data e hora) para o formato data (trazendo apenas datas). O mesmo foi feito para a coluna de data de nascimento. A coluna monthyear foi desconsiderada, uma vez que trazia a mesma informação que a coluna de data de saída. Para esse tratamento usei o "CAST"
-- Tratei a coluna "sex_upon_outcome" para trazer de forma separada as informações de sexo e castração do animal. Para isso usei "CASE".

**Camada Ouro**

In [0]:
-- Para não poluir a análise valores como "Outros" e "Unknown" foram desconsiderados

--- Intakes

DROP TABLE IF EXISTS ouro_aac_intakes;
CREATE TABLE ouro_aac_intakes AS
SELECT * FROM prata_aac_intakes
WHERE especie != 'Other'
AND sexo != 'Unknown'
AND castrado != 'Unknown'
AND condicao_entrada != 'Other';

--- Outcomes

DROP TABLE IF EXISTS ouro_aac_outcomes;
CREATE TABLE ouro_aac_outcomes AS
SELECT * FROM prata_aac_outcomes
WHERE especie != 'Other'
AND sexo != 'Unknown'
AND castrado != 'Unknown'
AND tipo_saida != 'Unknown';



**Camada Ouro - aac_intakes**

In [0]:
-- Modelagem estrela - Intakes

-- Dimensões:
-- Dimensão animal: id_dim_animal (int - primary key); nome (char); especie (char); raca (char); sexo(char); castrado (char)
DROP TABLE IF EXISTS dim_animal;
CREATE TABLE dim_animal AS
SELECT
  nome_animal,
  especie,
  raca,
  cor,
  sexo,
  castrado,
  monotonically_increasing_id() AS id_dim_animal
FROM (SELECT DISTINCT nome_animal, especie, raca, cor, sexo, castrado
  FROM ouro_aac_intakes);

-- Dimensão animal: id_dim_tempo (int - primary key); data_entrada (date); ano_entrada (int); mes_entrada (int); dia_entrada (int)
DROP TABLE IF EXISTS dim_tempo;
CREATE TABLE dim_tempo AS
SELECT
  data_entrada,
  year(data_entrada) as ano_entrada,
  month(data_entrada) as mes_entrada,
  day(data_entrada) as dia_entrada,
  monotonically_increasing_id() AS id_dim_data
FROM (SELECT DISTINCT data_entrada
  FROM ouro_aac_intakes);

-- Dimensão local: id_dim_local (int - primary key); local_encontrado (char)
DROP TABLE IF EXISTS dim_local;
CREATE TABLE dim_local AS
SELECT
  local_encontrado,
  monotonically_increasing_id() AS id_dim_local
FROM (SELECT DISTINCT local_encontrado
  FROM ouro_aac_intakes);

-- Dimensão local: id_dim_condicao (int - primary key); condicao_entrada (char)
DROP TABLE IF EXISTS dim_condicao;
CREATE TABLE dim_condicao AS
SELECT
  condicao_entrada,
  monotonically_increasing_id() AS id_dim_condicao
FROM (SELECT DISTINCT condicao_entrada
  FROM ouro_aac_intakes);

-- Dimensão tipo: id_dim_tipo (int - primary key); tipo_entrada (char)
DROP TABLE IF EXISTS dim_tipo;
CREATE TABLE dim_tipo AS
SELECT
  tipo_entrada,
  monotonically_increasing_id() AS id_dim_tipo
FROM (SELECT DISTINCT tipo_entrada
  FROM ouro_aac_intakes);

-- Tabela Fato
DROP TABLE IF EXISTS fato_entrada;
CREATE TABLE fato_entrada AS
SELECT
  o.id AS id_fato_entrada,
  o.idade_entrada,

  da.id_dim_animal,
  dt.id_dim_data,
  dl.id_dim_local,
  dc.id_dim_condicao,
  dti.id_dim_tipo

FROM ouro_aac_intakes o

LEFT JOIN dim_animal da
  ON o.nome_animal = da.nome_animal
 AND o.especie     = da.especie
 AND o.raca        = da.raca
 AND o.cor         = da.cor
 AND o.sexo        = da.sexo
 AND o.castrado    = da.castrado

LEFT JOIN dim_tempo dt
  ON o.data_entrada = dt.data_entrada

LEFT JOIN dim_local dl
  ON o.local_encontrado = dl.local_encontrado

LEFT JOIN dim_condicao dc
  ON o.condicao_entrada = dc.condicao_entrada

LEFT JOIN dim_tipo dti
  ON o.tipo_entrada = dti.tipo_entrada;

**Camada Ouro - acc_outcomes**

In [0]:
-- Modelagem estrela - Outcomes

-- Dimensões:
-- Dimensão animal: id_dim_animal (int - primary key); nome (char); especie (char); raca (char); sexo(char); castrado (char)
DROP TABLE IF EXISTS dim_animal_saida;
CREATE TABLE dim_animal_saida AS
SELECT
  nome_animal,
  data_nascimento,
  especie,
  raca,
  cor,
  sexo,
  castrado,
  monotonically_increasing_id() AS id_dim_animal_saida
FROM (SELECT DISTINCT nome_animal, data_nascimento, especie, raca, cor, sexo, castrado
  FROM ouro_aac_outcomes);

-- Dimensão animal: id_dim_tempo (int - primary key); data_entrada (date); ano_entrada (int); mes_entrada (int); dia_entrada (int)
DROP TABLE IF EXISTS dim_tempo_saida;
CREATE TABLE dim_tempo_saida AS
SELECT
  data_saida,
  year(data_saida) as ano_saida,
  month(data_saida) as mes_saida,
  day(data_saida) as dia_saida,
  monotonically_increasing_id() AS id_dim_data_saida
FROM (SELECT DISTINCT data_saida
  FROM ouro_aac_outcomes);

-- Dimensão tipo: id_dim_tipo (int - primary key); tipo_entrada (char)
DROP TABLE IF EXISTS dim_tipo_saida;
CREATE TABLE dim_tipo_saida AS
SELECT
  tipo_saida,
  subtipo_saida,
  monotonically_increasing_id() AS id_dim_tipo_saida
FROM (SELECT DISTINCT tipo_saida, subtipo_saida
  FROM ouro_aac_outcomes);

-- Tabela Fato
DROP TABLE IF EXISTS fato_saida;
CREATE TABLE fato_saida AS
SELECT
  os.id AS id_fato_saida,
  os.idade_saida,

  das.id_dim_animal_saida,
  dts.id_dim_data_saida,
  dtis.id_dim_tipo_saida

FROM ouro_aac_outcomes os

LEFT JOIN dim_animal_saida das
  ON os.nome_animal = das.nome_animal
 AND os.data_nascimento     = das.data_nascimento
 AND os.especie     = das.especie
 AND os.raca        = das.raca
 AND os.cor         = das.cor
 AND os.sexo        = das.sexo
 AND os.castrado    = das.castrado

LEFT JOIN dim_tempo_saida dts
  ON os.data_saida = dts.data_saida

LEFT JOIN dim_tipo_saida dtis
  ON os.tipo_saida = dtis.tipo_saida
  AND os.subtipo_saida = dtis.subtipo_saida;